In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

# Garder la page ouverte
chrome_options = Options() 
chrome_options.add_experimental_option("detach", True)

# On recupere le controle du driver
DRIVER_PATH = "./chromedriver"
driver = webdriver.Chrome(DRIVER_PATH, options=chrome_options)
driver.implicitly_wait(5)

driver.get("https://www.bienici.com/recherche/achat/hauts-de-seine-92/appartement?prix-max=300000")
try:
    driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div/div/div[3]/button[2]").click()
except:
    pass

h2 = driver.find_element(By.XPATH, "/html/body/div[2]/div[3]/div[3]/div/div[1]/div[3]/div[3]/div[2]/div/span/h2").text
nbr_element = int(h2.replace(" ", "")[:4])
print(nbr_element)

nbr_page = int((nbr_element - (nbr_element % 24)) /24)
nbr_element_derniere_page = nbr_element % 24
print(nbr_page)
print(nbr_element_derniere_page)

CodePostal = []
Prix = []
PrixMetreCarre = []
MetreCarre = []
NbrPiece = []
cpt = 0
nbr = 24

for i in range(1, nbr_page+2):
#for i in range(1, 7):
#for i in range(nbr_page-1, nbr_page+1):
    try:
        driver.get("https://www.bienici.com/recherche/achat/hauts-de-seine-92/appartement?prix-max=300000&page=" + str(i))
        time.sleep(1)
        try:
            driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div/div/div[3]/button[2]").click()
        except:
            pass
    except:
        print(i, y, "pas ok")

    if (i == nbr_page+1):
        nbr = nbr_element_derniere_page 

    for y in range(1, nbr+1):
        try: # Popup chiant
            driver.find_element(By.XPATH, "/html/body/div[2]/div[30]/div/div/div/div/button").click()
            time.sleep(1)
        except:
            pass

        try: # 2e Popup chiant
            driver.find_element(By.XPATH, "/html/body/div[2]/div[30]/div/div/div[2]/button").click()
            time.sleep(1)
        except:
            pass

        try :
            driver.find_element(By.XPATH, f"/html/body/div[2]/div[3]/div[3]/div/div[2]/section/div/div[1]/article["+ str(y) +"]/div/div/div/div[1]/div/div[2]/div/div[2]/a").click()
            time.sleep(1)
            try:
                codepostal = driver.find_element(By.XPATH, "/html/body/div[2]/section/div/div[3]/div[1]/div[1]/h1/span").text
                codepostal = int(codepostal.replace(" ", "")[:5])
            except:
                pass
            try:
                prix = driver.find_element(By.XPATH, "/html/body/div[2]/section/div/div[3]/div[1]/div[1]/div[2]/div[1]/div[1]/div/div/span[1]").text
                prix = int(prix.replace(" ","")[:-1])
            except:
                pass
            try:
                prixmetrecarre = driver.find_element(By.XPATH,"/html/body/div[2]/section/div/div[3]/div[1]/div[1]/div[2]/div[1]/div[1]/div/div/span[2]").text
                try:
                    prixmetrecarre = int(prixmetrecarre.replace(" ","")[:-4])
                except:
                    prixmetrecarre = int(prixmetrecarre.replace(" ","")[:-5].replace(",",""))
                    prixmetrecarre *= 100 
            except:
                pass
            try:
                metrecarre = driver.find_element(By.XPATH, "/html/body/div[2]/section/div/div[3]/section[3]/div[1]/div[4]/span").text
                metrecarre = float(metrecarre.replace(" ","")[:-2].replace(",","."))
            except:
                pass
            try:
                nbrpiece = driver.find_element(By.XPATH, "/html/body/div[2]/section/div/div[3]/section[3]/div[1]/div[5]/span").text
                try:
                    nbrpiece = int(nbrpiece.replace(" ","")[:-5])
                except:
                    nbrpiece = int(nbrpiece.replace(" ","")[:-6]) 
            except:
                pass
            #print(codepostal, prix, prixmetrecarre, metrecarre, nbrpiece)

            CodePostal.append(codepostal)
            Prix.append(prix)
            PrixMetreCarre.append(prixmetrecarre)
            MetreCarre.append(metrecarre)
            NbrPiece.append(nbrpiece)

            #print(i, y, "ok")
            time.sleep(1)
            cpt += 1
            driver.back()
        except:
            print(i, y, "pas ok", 1)


driver.close()
driver.quit()
print(cpt)

df = pd.DataFrame({'Code Postal': CodePostal, 'Prix (€)': Prix, 'Surface (m2)': MetreCarre,'Prix du m2': PrixMetreCarre, 'Nbr de Pieces':NbrPiece})
df.to_csv('BienICI2.csv', index=False, encoding='utf-8')

# 504m.4.0s

/var/folders/1_/54_5j3cn25jd5mt0cyzggqj80000gn/T/ipykernel_43524/1613127917.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(DRIVER_PATH, options=chrome_options)


2102
87
14
144


In [ ]:
# Merge

import os, glob
import pandas as pd

path = "/Users/yann/Desktop/Ydays/Data/bien_ici/" # full path des fichiers csv
#all_files = glob.glob(path + "A*.csv") # on fait des trucs grace au package globe
all_files = glob.glob("Bien*.csv")
#print(all_files) # test

all_df = []
for f in all_files:
    df = pd.read_csv(f, sep=',')
    df['file'] = f.split('/')[-1]
    all_df.append(df)

# print(len(all_df))
# print(all_df[2])

merged_df = pd.concat(all_df, ignore_index=True) # concatener plusieurs csv
#merged_df.drop_duplicates() # enlever les doublons 
merged_df.drop('file', inplace=True, axis=1) # supprimer la colonne 'file' (de quelle fichier provient la ligne)
merged_df.drop_duplicates() # enlever les doublons 
#merged_df.drop('Unnamed', inplace=True, axis=1) # supprimer la colonne '' (Il y a des sort d'indices; jsp d'ou ils viennent)
merged_df.to_csv("BIEN_ICI.csv") # tout mettre dans un csv